In [ ]:
%pip install pandas matplotlib seaborn numpy

# Transaction Field Analysis

## Field Descriptions and Examples

| Category | Field | Description | Example |
|----------|-------|-------------|---------|
| Block Related | `block_hash` | Hash of the block containing this transaction | `0x8cade31fe32dfcb591168e327b05f1f0138279f89d2735b7262a16526785fea3` |
| | `block_number` | Block number where this transaction was included | `26484602` |
| Transaction Identifiers | `hash` | Unique hash of the transaction | `Not shown in example` |
| | `transaction_index` | Position of transaction in the block (0-based index) | `Not shown in example` |
| | `nonce` | Sender's transaction count (prevents double-spending) | `Not shown in example` |
| Addresses | `from` | Address initiating the transaction | `0xdeaddeaddeaddeaddeaddeaddeaddeaddead0001` |
| | `to` | Recipient address (null for contract creation) | `Not shown in example` |
| Value & Data | `value` | Amount of ETH transferred in Wei (0x00 means no ETH transfer) | `0x00` |
| | `input` | Transaction input data (contract interaction data or deployment code) | `Not shown in example` |
| Gas Related | `gas` | Maximum gas units the transaction can use | `0x0f4240` |
| | `gas_price` | Price per gas unit the sender is willing to pay | `0x00` |
| | `gas_used` | Actual gas units consumed by the transaction | `Not shown in example` |
| | `effective_gas_price` | Actual price per gas unit paid | `Not shown in example` |
| | `cumulative_gas_used` | Total gas used in the block up to this transaction | `Not shown in example` |
| EIP-1559 Fields | `max_priority_fee_per_gas` | Max additional fee per gas to be paid to miners (null for legacy tx) | `0x52d80c1f` |
| | `max_fee_per_gas` | Maximum total fee per gas (null for legacy tx) | `0x52f4e652` |
| Other | `chain_id` | Network identifier (0x2105 for Base) | `0x2105` |
| | `status` | Transaction status (1 = success, 0 = failure) | `1` |

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
def load_and_preprocess_data():
    df = pd.read_parquet("../data/transactions.parquet")
    
    # Convert hex values to numeric
    hex_columns = [
        "gas", "gas_price", "max_priority_fee_per_gas", "max_fee_per_gas",
        "cumulative_gas_used", "effective_gas_price", "gas_used", "nonce",
        "value"  # Added value to conversion
    ]

    for col in hex_columns:
        df[col] = df[col].apply(lambda x: int(x, 16) if isinstance(x, str) and x.startswith("0x") else x)

    # Convert chain_id to integer where possible
    df["chain_id"] = df["chain_id"].apply(
        lambda x: int(x, 16) if isinstance(x, str) and x.startswith("0x") else x
    )

    # Ensure numeric types
    df["transaction_index"] = df["transaction_index"].astype(int)
    df["block_number"] = df["block_number"].astype(int)
    df["value"] = pd.to_numeric(df["value"], errors='coerce')
    
    print("Dataset Info:")
    print(df.info())
    print("\nMissing Values:")
    print(df.isnull().sum())
    
    # Print value ranges for key metrics
    print("\nValue Ranges for Key Metrics:")
    numeric_cols = ['gas_price', 'gas_used', 'effective_gas_price', 'value']
    print(df[numeric_cols].describe())
    
    return df

In [5]:
# Main analysis cell to tun the functions
df = load_and_preprocess_data()


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13471618 entries, 0 to 13471617
Data columns (total 18 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   block_hash                object 
 1   block_number              int64  
 2   from                      object 
 3   gas                       int64  
 4   gas_price                 int64  
 5   hash                      object 
 6   input                     object 
 7   nonce                     int64  
 8   to                        object 
 9   transaction_index         int64  
 10  value                     float64
 11  max_priority_fee_per_gas  float64
 12  max_fee_per_gas           float64
 13  chain_id                  float64
 14  cumulative_gas_used       int64  
 15  effective_gas_price       int64  
 16  gas_used                  int64  
 17  status                    uint8  
dtypes: float64(4), int64(8), object(5), uint8(1)
memory usage: 1.7+ GB
None

Missing Value

In [ ]:

# # 1. Load and Inspect the Data
# def load_and_preprocess_data():
#     df = pd.read_parquet("../data/transactions.parquet")
    
#     # List of columns that are in hex format
#     hex_columns = [
#         "gas", "gas_price", "max_priority_fee_per_gas", "max_fee_per_gas",
#         "cumulative_gas_used", "effective_gas_price", "gas_used", "nonce"
#     ]

#     # Convert hex to integer (ignoring null values)
#     for col in hex_columns:
#         df[col] = df[col].apply(lambda x: int(x, 16) if isinstance(x, str) and x.startswith("0x") else x)

#     # Convert transaction_index & block_number to integers if they aren't already
#     df["transaction_index"] = df["transaction_index"].astype(int)
#     df["block_number"] = df["block_number"].astype(int)

#     # Re-check data types
#     print(df.dtypes)
    
#     # Print data info
#     print("Dataset Info:")
#     print(df.info())
#     print("\nMissing Values:")
#     print(df.isnull().sum())
#     return df

# # 2. Handle Missing Values (as per GAS_PRICE_PREDICTION.md)
# def handle_missing_values(df):
#     df['max_priority_fee_per_gas'] = df['max_priority_fee_per_gas'].fillna(0)
#     df['max_fee_per_gas'] = df['max_fee_per_gas'].fillna(0)
#     df['effective_gas_price'] = df['effective_gas_price'].fillna(df['gas_price'])
#     return df

# # 3. Data Visualization for Gas Price Prediction
# def visualize_gas_metrics(df):
#     # 1. Gas Price Distribution
#     plt.figure(figsize=(12, 6))
#     plt.hist(df['gas_price'], bins=50, alpha=0.7)
#     plt.title('Gas Price Distribution')
#     plt.xlabel('Gas Price (Wei)')
#     plt.ylabel('Frequency')
#     plt.yscale('log')  # Log scale for better visualization
#     plt.show()

#     # 2. Gas Price by Block Number (Time Series)
#     plt.figure(figsize=(12, 6))
#     df.groupby('block_number')['gas_price'].mean().plot()
#     plt.title('Average Gas Price per Block')
#     plt.xlabel('Block Number')
#     plt.ylabel('Average Gas Price (Wei)')
#     plt.show()

#     # 3. Block Congestion Analysis
#     plt.figure(figsize=(12, 6))
#     plt.scatter(df['cumulative_gas_used'], df['gas_price'], alpha=0.5, s=5)
#     plt.title('Block Congestion vs Gas Price')
#     plt.xlabel('Cumulative Gas Used')
#     plt.ylabel('Gas Price (Wei)')
#     plt.show()

#     # 4. Gas Usage Patterns
#     plt.figure(figsize=(12, 6))
#     plt.scatter(df['gas_used'], df['gas_price'], alpha=0.5, s=5)
#     plt.title('Gas Used vs Gas Price')
#     plt.xlabel('Gas Used')
#     plt.ylabel('Gas Price (Wei)')
#     plt.show()

# # 4. Feature Engineering for Gas Price Prediction
# def engineer_features(df):
#     # Block-based Features
#     df['gas_price_last_5_avg'] = df.groupby('block_number')['gas_price'].transform(
#         lambda x: x.rolling(window=5, min_periods=1).mean()
#     )
    
#     # Gas Usage Features
#     df['gas_utilization_ratio'] = df['gas_used'] / df['gas']
#     df['block_congestion'] = df['cumulative_gas_used'] / df['gas'].max()
    
#     # Priority Fee Features
#     df['priority_fee_ratio'] = df['max_priority_fee_per_gas'] / (df['gas_price'] + 1e-10)
#     df['fee_premium'] = df['gas_price'] - df['effective_gas_price']
    
#     return df

# # Main analysis
# df = load_and_preprocess_data()
# df = handle_missing_values(df)
# # visualize_gas_metrics(df)
# # df = engineer_features(df)

# # Display summary statistics for gas price prediction
# print("\nSummary Statistics for Gas Price Prediction:")
# prediction_features = [
#     'gas_price', 'gas_used', 'gas_utilization_ratio',
#     'block_congestion', 'priority_fee_ratio', 'fee_premium'
# ]
# print(df[prediction_features].describe())

# # Display correlations
# plt.figure(figsize=(12, 8))
# sns.heatmap(df[prediction_features].corr(), annot=True, cmap='coolwarm')
# plt.title('Feature Correlations for Gas Price Prediction')
# plt.show()